### This is the actual code to run

In [ ]:
import fitz  # PyMuPDF
import base64
import tkinter as tk
from tkinter import filedialog, messagebox
import requests
import re
import json
import os  # At the top of your script
import threading
import queue
import time
import pandas as pd
from datetime import datetime
from PIL import Image
from tkinter import ttk
import io
import math
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment variable
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    raise ValueError("OpenAI API key not found. Please set OPENAI_API_KEY in your .env file")


class FileSelectionDialog:
    def close_gui(self):
        """Safely close the GUI from the main thread."""
        self.root.destroy()

    def __init__(self, root):
        self.root = root
        self.root.title("File Selections")
        self.update_queue = queue.Queue()  # Queue to hold update tasks

        # Initialize variables
        self.pdf_path = None
        self.report_path = None
        self.total_pages = 0
        self.max_pages = 1000000  ##Incase you want to test you can lower this variable
        self.class_name = None
        self.subject_name = None
        self.start_page = 1  # Default start page
        self.stop_page = None

        self.setup_widgets()
        self.poll_queue()  # Start polling the queue for updates

    def poll_queue(self):
        """Check the queue for new messages"""
        try:
            while True:
                message = self.update_queue.get_nowait()
                if message == ("close",):  # Check for the close signal
                    self.close_gui()
                else:
                    current_page, total_pages = message
                    self.update_progress(current_page, total_pages)
        except queue.Empty:
            pass
        finally:
            # Schedule another call to this method if the root is still alive
            if self.root.winfo_exists():
                self.root.after(100, self.poll_queue)

    def safe_update_progress(self, current_page, total_pages):
        """Enqueue the progress update."""
        self.update_queue.put((current_page, total_pages))

    def setup_widgets(self):
        # Add class selection
        tk.Label(self.root, text="Enter Class Name:").pack(padx=20, pady=5)
        self.class_entry = tk.Entry(self.root)
        self.class_entry.pack(pady=(0, 10))

        # Add subject selection
        tk.Label(self.root, text="Select Subject:").pack(padx=20, pady=5)
        self.subject_combo = ttk.Combobox(
            self.root,
            values=["SST", "Mathematics", "Science", "English", "CRE"],
            state="readonly",
        )  # readonly prevents typing
        self.subject_combo.pack(pady=(0, 20))
        self.subject_combo.set("SST")

        tk.Label(self.root, text="Select the source PDF file:").pack(padx=20, pady=5)
        self.pdf_button = tk.Button(
            self.root, text="Choose PDF", command=self.select_pdf_file
        )
        self.pdf_button.pack(pady=(0, 20))

        tk.Label(
            self.root,
            text="Select the path and set a name for saving the Excel report:",
        ).pack(padx=20, pady=5)
        self.report_button = tk.Button(
            self.root, text="Choose Report Path", command=self.select_output_path
        )
        self.report_button.pack(pady=(0, 20))

        start_page_frame = tk.Frame(self.root)
        start_page_frame.pack(pady=10)

        tk.Label(start_page_frame, text="Start from page (optional):").pack(
            side=tk.LEFT, padx=5
        )
        self.start_page_entry = tk.Entry(start_page_frame, width=10)
        self.start_page_entry.pack(side=tk.LEFT, padx=5)
        self.start_page_entry.insert(0, "1")  # Default value

        stop_page_frame = tk.Frame(self.root)
        stop_page_frame.pack(pady=10)

        tk.Label(stop_page_frame, text="Stop at page (optional):").pack(
            side=tk.LEFT, padx=5
        )
        self.stop_page_entry = tk.Entry(stop_page_frame, width=10)
        self.stop_page_entry.pack(side=tk.LEFT, padx=5)

        # Add validation for numeric input
        def validate_page_number(P):
            if P == "":
                return True
            return P.isdigit() and int(P) > 0

        vcmd = (self.root.register(validate_page_number), "%P")
        self.start_page_entry.config(validate="key", validatecommand=vcmd)

        # Label to display the selected directory and the default filename
        self.report_path_label = tk.Label(self.root, text="")
        self.report_path_label.pack(pady=(5, 10))  # Adjust padding as necessary

        self.confirm_button = tk.Button(
            self.root, text="Confirm Selections", command=self.confirm_selections
        )
        self.confirm_button.pack(pady=20)

        # Progress label
        self.progress_label = tk.Label(self.root, text="")
        self.progress_label.pack(pady=10)

        # New progress label for misconceptions analysis
        self.analysis_progress_label = tk.Label(self.root, text="")
        self.analysis_progress_label.pack(pady=10)

    def select_pdf_file(self):
        file_path = filedialog.askopenfilename(
            title="Select the source PDF File", filetypes=[("PDF Files", "*.pdf")]
        )
        if file_path:
            self.pdf_path = file_path
            self.pdf_button.config(text=f"Selected: {file_path.split('/')[-1]}")

            # Get the total number of pages in the PDF
            try:
                doc = fitz.open(self.pdf_path)
                self.total_pages = len(doc)
                doc.close()
                self.progress_label.config(
                    text=f"Document has {self.total_pages} pages."
                )
            except Exception as e:
                messagebox.showerror("Error", f"Failed to read the PDF: {e}")
                self.pdf_path = None
                self.total_pages = 0

    def select_output_path(self):
        directory = filedialog.askdirectory(
            title="Select the directory to save the report"
        )
        if directory:
            current_time = datetime.now()
            formatted_time = current_time.strftime("%d_%m_%Y_%H_%M")
            class_name = self.class_entry.get().strip().replace(" ", "_")
            subject_name = self.subject_combo.get().strip().replace(" ", "_")

            default_filename = (
                f"exam_report_{class_name}_{subject_name}_{formatted_time}.xlsx"
            )
            self.report_path = os.path.join(directory, default_filename)
            self.report_directory = directory  # Save the directory if needed elsewhere
            self.report_button.config(text=f"Path Selected: {directory}")
            self.report_path_label.config(
                text=f"Directory: {directory}\nFile will be saved as: {default_filename}"
            )

    def confirm_selections(self):
        """Handle the confirmation of file selections."""
        self.class_name = self.class_entry.get().strip()
        self.subject_name = self.subject_combo.get().strip()

        # Get and validate start page
        try:
            start_page = int(self.start_page_entry.get())
            stop_page_text = self.stop_page_entry.get().strip()
            stop_page = int(stop_page_text) if stop_page_text else self.total_pages

            if start_page < 1 or (self.total_pages and start_page > self.total_pages):
                messagebox.showerror(
                    "Error", f"Start page must be between 1 and {self.total_pages}"
                )
                return

            if stop_page < start_page or (self.total_pages and stop_page > self.total_pages):
                messagebox.showerror(
                    "Error",
                    f"Stop page must between {start_page} and {self.total_pages}",
                )
                return
            
            self.start_page = start_page
            self.stop_page = stop_page

        except ValueError:
            if stop_page_text and not stop_page_text.isdigit():
                messagebox.showerror("Error", "Stop page must be a valid number")
                return
            self.start_page = 1  # Default to first page if invalid input
            self.stop_page = self.total_pages

        if (
            not self.pdf_path
            or not self.report_path
            or not self.class_name
            or not self.subject_name
        ):
            messagebox.showerror(
                "Error",
                "Please fill in all fields (PDF, Report path, Class, and Subject)",
            )
        else:
            self.pdf_button.config(state=tk.DISABLED)
            self.report_button.config(state=tk.DISABLED)
            self.class_entry.config(state=tk.DISABLED)
            self.subject_combo.config(state=tk.DISABLED)
            self.start_page_entry.config(state=tk.DISABLED)
            self.stop_page_entry.config(state=tk.DISABLED)
            self.confirm_button.config(state=tk.DISABLED)
            print("Selections confirmed. Ready for processing.")
            self.root.quit()  # Quit the Tkinter event loop

    def reset_selections(self):
        self.pdf_path = None
        self.report_path = None
        self.total_pages = 0
        self.pdf_button.config(text="Choose PDF")
        self.report_button.config(text="Choose Report Path")
        self.progress_label.config(text="")

    def update_progress(self, current_page, total_pages):
        """Update the progress label dynamically."""
        stop_page = self.stop_page if self.stop_page else total_pages
        self.progress_label.config(
            text=f"Processing page {current_page + 1}/{stop_page} of {total_pages} pages..."
        )
        self.root.update_idletasks()

    def update_analysis_progress(self, current_item, total_items):
        """Update the analysis progress label directly called by the GUI thread."""
        self.analysis_progress_label.config(
            text=f"Analysing responses {current_item}/{total_items}..."
        )
        self.root.update_idletasks()

    def update_topic_progress(self, current_question, total_questions):
        """Update the topic analysis progress label directly called by the GUI thread."""
        self.analysis_progress_label.config(
            text=f"Categorizing topics: Question {current_question}/{total_questions}..."
        )
        self.root.update_idletasks()


def encode_image_to_base64(pixmap):
    """Encode a PyMuPDF pixmap to a base64 string."""
    img_bytes = pixmap.tobytes("png")
    return base64.b64encode(img_bytes).decode("utf-8")


def robust_process_image(base64_image, last_known_student_name, retries=6, delay=16):
    attempt = 0
    last_exception = None
    while attempt < retries:
        response_content = None
        try:
            response_content = process_image(base64_image, last_known_student_name)
            if not (
                isinstance(response_content, dict)
                and "studentName" in response_content
                and "entries" in response_content
            ):
                raise KeyError("Response does not conform to the expected JSON schema.")
            if response_content:
                return response_content, None
        except (requests.exceptions.RequestException, json.JSONDecodeError, Exception) as e:
            last_exception = e
            print(f"Request failed with error: {e}. Retrying...")
            time.sleep(delay)
            attempt += 1
        except (KeyError) as e:
            last_exception = e
            print("Key error:", e)
            print("Response was:", response_content)
            print("Retrying...")
            time.sleep(delay)
            attempt += 1
        else:
            try:
                if (
                    "error" in response_content
                    and response_content["error"]["message"]
                    == "The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently."
                ):
                    print("Model produced invalid content error received. Retrying...")
                    time.sleep(delay)
                    attempt += 1
                elif (
                    "error" in response_content
                    and response_content["error"]["message"]
                    == "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp']."
                ):
                    print("Unsupported image format or size error received. Retrying...")
                    time.sleep(delay)
                    attempt += 1
            except (TypeError, KeyError):
                print("Unexpected response format or missing data. Retrying...")
                time.sleep(delay)
                attempt += 1
    print(f"robust_process_image: failed after {retries} retries. Last exception: {last_exception}")
    return None, last_exception


def save_page_image(base64_image, original_file_name, page_number, report_directory):
    """
    Save the page image to a 'page-pictures' subdirectory within the report directory.

    Args:
        base64_image: The base64 encoded image data
        original_file_name: Name of the original PDF file
        page_number: Current page number
        report_directory: The directory where the report will be saved
    """
    # Create page-pictures folder within the report directory
    output_folder = os.path.join(report_directory, "page_pictures")

    # Create the directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Format the filename
    image_filename = f"{original_file_name}_page{page_number}.png"

    try:
        # Decode the base64 string to bytes
        image_data = base64.b64decode(base64_image)
        image = Image.open(io.BytesIO(image_data))

        # Save the image
        save_path = os.path.join(output_folder, image_filename)
        image.save(save_path)
        print(f"Image saved to {save_path}")

    except Exception as e:
        print(f"Error saving image: {e}")


def process_image(base64_image, last_known_student_name):
    """Send the base64 image to OpenAI, returning the parsed content with a dynamic prompt."""

    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {OPENAI_API_KEY}"}
    prompt_text = (
        f"In this image, first identify if there is a field at the top of the page explicitly "
        f"labelled 'Name' followed by a colon or on the right of a label 'Name:' on the same line. "
        f"If such a label exists and a name immediately follows it on the same line, extract that "
        f"name and assign it as the value of 'studentName' in the output JSON object. "
        f"If the Name label is not followed by a name on the same line or if the label does not "
        f"exist, use '{last_known_student_name}' as the studentName. For the rest of the content, "
        f"transcribe only the exam questions into an array of entries. Each entry must be a JSON "
        f"object with exactly the following keys: 'questionNo', 'question', 'answer', and 'grading'. "
        f"The output must be a valid JSON object with the following structure:\n"
        f'{{ "studentName": "<Extracted or default name>", "entries": [ {{ "questionNo": "<Question number>", '
        f'"question": "<Question text>", "answer": "<Answer text>", "grading": "<Correct/Incorrect/Not Graded or '
        f'empty string>" }}, ... ] }} '
        f"For questions with multiple parts (e.g., 36(a)), if there are multiple answers, produce a separate entry "
        f"for each answer, repeating the question text for each part. If a question provides an option to answer "
        f"either one part or the other, transcribe only the part that was answered. If the student hasn't answered, "
        f"leave the 'answer' field empty. Grade each answer as 'Correct' if there is a red tick mark. If an answer "
        f"has any marks other than a red tick mark and does not have a red tick mark, treat it as 'Incorrect'. "
        f"If an answer contains a red tick mark along with other marks, treat it as 'Correct'. If no mark is present "
        f"at all, grade it as 'Not Graded'. Ensure no extra spaces are added at the beginning or end of any text values, "
        f"and ignore any non-exam instructions. Your response must be a single, valid JSON object and nothing else. Do not wrap it in markdown code blocks."
    )

    payload = {
        "model": "ft:gpt-4o-2024-08-06:personal:my-answersheet-experiment-28-12-2024:AjKchLD0",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "high",
                        },
                    },
                ],
            }
        ],
        "max_tokens": 1500,
        "response_format": {"type": "json_object"},
    }
    response = requests.post(
        "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
    )
    print("Prompt text:")
    print(prompt_text)
    print("\nRaw API response text:")
    print(response.text)
    response.raise_for_status()  # Raise an error for bad status codes
    # Get the raw content from the assistant
    raw_content = response.json()["choices"][0]["message"]["content"]
    print("Raw content from API:")
    print(raw_content)

    # Attempt to parse the JSON directly first
    try:
        parsed_response = json.loads(raw_content)
        print("\nParsed JSON object directly:")
        print(parsed_response)
        return parsed_response
    except json.JSONDecodeError:
        print("Direct JSON parsing failed. Attempting to extract from markdown.")
        # Fallback: Extract JSON from markdown code block if present
        match = re.search(r"```json\s*(\{.*?\})\s*```", raw_content, re.DOTALL)
        if match:
            json_str = match.group(1)
            try:
                parsed_response = json.loads(json_str)
                print("\nParsed JSON object from markdown:")
                print(parsed_response)
                return parsed_response
            except json.JSONDecodeError as e:
                print("\nJSONDecodeError after extracting from markdown:")
                print(e)
                raise e  # Re-raise the specific error
        
        # Fallback 2: Find the first and last curly brace
        start = raw_content.find("{")
        end = raw_content.rfind("}") + 1
        if start != -1 and end > start:
            json_str = raw_content[start:end]
            try:
                parsed_response = json.loads(json_str)
                print("\nParsed JSON object using brace finding:")
                print(parsed_response)
                return parsed_response
            except json.JSONDecodeError as e:
                print("\nJSONDecodeError after finding braces:")
                print(e)
                raise e

        # If all parsing attempts fail, raise an error
        raise json.JSONDecodeError("Failed to find or parse JSON in the response.", raw_content, 0)


def get_question_numbers_from_json(entries):
    """
    entries is a list of dicts, each dict representing one question item, e.g.:
        {
          "questionNo": "45(a)",
          "question": "Write two steps...",
          "answer": "Evicting people...",
          "grading": "Correct"
        }
    We want to find the first and last fully numeric questionNo (e.g. '45').
    """

    first_question_number = "N/A"
    last_question_number = "N/A"

    question_numbers = []
    for item in entries:
        # Get the question number field, e.g. "45(a)"
        q_number = item.get("questionNo", "")
        # Remove parentheses parts like (a), (i) etc.
        q_number_cleaned = re.sub(r"\(.*?\)", "", q_number).strip()
        question_numbers.append(q_number_cleaned)

    # Now filter to numeric question numbers, e.g. "45"
    numeric_question_numbers = [num for num in question_numbers if num.isdigit()]
    if numeric_question_numbers:
        first_question_number = numeric_question_numbers[0]
        last_question_number = numeric_question_numbers[-1]

    return first_question_number, last_question_number


def process_pdf(
    dialog,
    doc,
    last_question,
    current_student_name,
    csv_file_path,
    jsonl_file_path,
    start_page=1,
    stop_page=None,
):
    start_idx = start_page - 1
    stop_idx = stop_page if stop_page else len(doc)
    original_file_name = os.path.basename(dialog.pdf_path).replace(".pdf", "")

    failed_pages = []  # To keep track of skipped pages and reasons

    with open(csv_file_path, mode="w", encoding="utf-8") as csv_file, open(
        jsonl_file_path, mode="w", encoding="utf-8"
    ) as jsonl_file:

        csv_file.write(
            "Student Name,Question No,Question,Answer,Grading,PageNumber,ClassName,SubjectName\n"
        )

        num_pages_to_process = min(stop_idx, dialog.max_pages)
        className = str(dialog.class_name).strip()
        subjectName = str(dialog.subject_name).strip()

        for page_num in range(start_idx, num_pages_to_process):
            dialog.safe_update_progress(page_num, dialog.total_pages)
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            base64_image = encode_image_to_base64(pix)

            save_page_image(
                base64_image, original_file_name, page_num + 1, dialog.report_directory
            )

            print(f"Processing page {page_num + 1}/{dialog.total_pages}...")
            response_json, last_exception = robust_process_image(base64_image, current_student_name)
            if response_json is None:
                print(f"Failed to process page {page_num + 1} after retries. Skipping.")
                failed_pages.append({
                    "page": page_num + 1,
                    "reason": str(last_exception)
                })
                continue

            student_name_extracted = response_json.get(
                "studentName", current_student_name
            )
            entries = response_json.get("entries", [])

            if student_name_extracted:
                current_student_name = student_name_extracted

            first_question, last_question = get_question_numbers_from_json(entries)
            print(f"First question number: {first_question}")
            print(f"Last question number: {last_question}")

            if entries:
                for entry in entries:
                    question_no = entry.get("questionNo", "")
                    question = entry.get("question", "")
                    answer = entry.get("answer", "")
                    grading = entry.get("grading", "")
                    page_str = str(page_num + 1)
                    question_escaped = question.replace('"', '""')
                    answer_escaped = answer.replace('"', '""')
                    grading_escaped = grading.replace('"', '""')
                    quoted_question = f'"{question_escaped}"'
                    quoted_answer = f'"{answer_escaped}"'
                    quoted_grading = f'"{grading_escaped}"'
                    quoted_class = f'"{className}"'
                    quoted_subject = f'"{subjectName}"'
                    csv_row = ",".join(
                        [
                            student_name_extracted,
                            question_no,
                            quoted_question,
                            quoted_answer,
                            quoted_grading,
                            page_str,
                            quoted_class,
                            quoted_subject,
                        ]
                    )
                    csv_file.write(csv_row + "\n")
                    jsonl_obj = {
                        "studentName": student_name_extracted,
                        "questionNo": question_no,
                        "question": question,
                        "answer": answer,
                        "grading": grading,
                        "pageNumber": page_num + 1,
                        "className": className,
                        "subjectName": subjectName,
                    }
                    jsonl_file.write(json.dumps(jsonl_obj) + "\n")
            else:
                csv_file.write(student_name_extracted + ",,,,\n")
                jsonl_file.write(
                    json.dumps({"studentName": student_name_extracted}) + "\n"
                )

    # === Report skipped/failed pages at the end ===
    if failed_pages:
        print("\nThe following pages failed after all retries and were skipped:")
        for fail in failed_pages:
            print(f"Page {fail['page']}: {fail['reason']}")
        # Optionally save to disk
        fail_path = csv_file_path.replace('.csv', '_failed_pages.json')
        with open(fail_path, "w") as f:
            json.dump(failed_pages, f, indent=2)
        print(f"\nFailed page log written to: {fail_path}")

def categorize_questions_by_topic(dialog, questions_df, output_path, subject_name="SST"):
    """
    Categorize questions by topic using OpenAI API in a batch.
    
    Args:
        questions_df: DataFrame containing questions
        output_path: Path to save the output Excel file
        subject_name: Name of the subject (e.g., 'SST')
        
    Returns:
        bool: True if successful, False if error occurred
    """
    try:
        print("DEBUG: questions_df shape:", questions_df.shape)
        print("DEBUG: questions_df columns:", questions_df.columns.tolist())
        print("DEBUG: questions_df head:")
        print(questions_df.head(10))

        # Define subject topics mapping
        subject_topics = {
            "SST": [
                "Physical and Human Geography",
                "Civics and Governance",
                "History and Heritage",
                "Economic Activities and Development",
                "Social Systems and Practices",
                "Environmental Conservation and Management",
            ],
            "Mathematics": [
                "Numbers and Numeration",
                "Basic Operations (Addition, Subtraction, Multiplication, Division)",
                "Fractions and Decimals",
                "Measurement (Length, Mass, Capacity, Time)",
                "Geometry and Shapes",
                "Money and Consumer Math",
                "Statistics and Data Handling",
                "Algebra and Patterns"
            ],
            "Science": [
                "Living Things and Life Processes",
                "Human Body Systems",
                "Plants and Animals",
                "Materials and Their Properties",
                "Energy and Forces",
                "Earth and Space",
                "Environmental Science",
                "Health and Safety"
            ],
            "English": [
                "Reading Comprehension",
                "Grammar and Language Structure",
                "Vocabulary Development",
                "Writing Skills",
                "Speaking and Listening",
                "Literature Appreciation",
                "Spelling and Punctuation",
                "Creative Writing"
            ],
            "CRE": [
                "Biblical Stories and Characters",
                "Christian Values and Morals",
                "Prayer and Worship",
                "Church History",
                "Religious Ceremonies and Celebrations",
                "Christian Living",
                "Biblical Geography",
                "Faith and Beliefs"
            ]
        }

        topics = subject_topics.get(subject_name, [])
        if not topics:
            print("Topic classification not available for this subject")
            questions_df["Topic"] = "Topic classification not available for this subject"
            questions_df["Confidence"] = 0
            questions_df["Explanation"] = "Topic classification not available for this subject"
            
            # Save and return early
            with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
                questions_df.to_excel(writer, sheet_name="Question Categories", index=False)
                pd.DataFrame().to_excel(writer, sheet_name="Topic Statistics")
            return True

        # Initialize empty lists to store all results
        all_topics = []
        all_confidences = []
        all_explanations = []
        
        chunk_size = 30
        total_questions = len(questions_df)
        total_batches = math.ceil(total_questions / chunk_size)

        for batch_num, i in enumerate(range(0, total_questions, chunk_size), 1):
            chunk = questions_df.iloc[i:i+chunk_size]
            end_idx = min(i + chunk_size, total_questions)
            
             # Update progress in dialog
            progress_msg = f"Categorizing batch {batch_num} of {total_batches}(Questions {i+1}-{end_idx} of {total_questions})"
            dialog.analysis_progress_label.config(text=progress_msg)
            dialog.root.update_idletasks()

            prompt = (
                f"Given the following questions from a {subject_name} exam, classify each into one of these topics:\n"
                f"{', '.join(topics)}\n\n"
                "Provide your response as a JSON array, where each object corresponds to a question and has these keys:\n"
                "- question_no: The question number\n"
                "- topic: The most relevant topic from the list above\n"
                "- confidence: A number between 0 and 1 indicating confidence in classification\n"
                "- explanation: A brief explanation of why this topic was chosen\n\n"
                "Questions:\n"
                
            )
            
            for idx, row in chunk.iterrows():
                prompt += f"- Question No: {row['Question No']}, Question: {row['Question']}\n"
            prompt += "\nOutput only the JSON array.\nLeave the Question No as given (e.g., '41(a)', '41(b)(ii)'), and do not convert them to plain integers."

            print("Prompt text:")
            print(prompt)
            
            
            try:
                headers = {
                    "Content-Type": "application/json",
                    "Authorization": f"Bearer {OPENAI_API_KEY}",
                }
                payload = {
                    "model": "gpt-4o-mini",
                    "messages": [{"role": "user", "content": prompt}],
                    "max_tokens": 2500,
                }
                
                response = requests.post(
                    "https://api.openai.com/v1/chat/completions",
                    headers=headers,
                    json=payload,
                )
                print("\nRaw API response text:")
                print(response.text)
                
                response.raise_for_status()
                content = response.json()["choices"][0]["message"]["content"]

                print("Raw content from API:")
                print(content)
                
                # Extract and parse JSON
                start = content.find("[")
                end = content.rfind("]") + 1
                if start == -1 or end <= start:
                    raise json.JSONDecodeError("No JSON array found in response", content, 0)
                
                json_str = content[start:end].strip()
                results = json.loads(json_str)
                
                # Process results for this chunk
                for idx, row in chunk.iterrows():
                    result = next(
                        (item for item in results if str(item.get("question_no")) == row["Question No"]),
                        None
                    )
                    if result:
                        all_topics.append(result.get("topic", ""))
                        all_confidences.append(result.get("confidence", 0))
                        all_explanations.append(result.get("explanation", ""))
                    else:
                        all_topics.append("Classification Failed")
                        all_confidences.append(0)
                        all_explanations.append("No matching question number found")
                
            except Exception as e:
                print(f"Error processing chunk {i//chunk_size + 1}: {str(e)}")
                # Add failed classifications for this chunk
                chunk_size = len(chunk)
                all_topics.extend(["Classification Failed"] * chunk_size)
                all_confidences.extend([0] * chunk_size)
                all_explanations.extend(["API error occurred"] * chunk_size)

        # Final progress update
        dialog.analysis_progress_label.config(text="Topic categorization completed!")
        dialog.root.update_idletasks()
        
        # Assign results to DataFrame
        questions_df["Topic"] = all_topics
        questions_df["Confidence"] = all_confidences
        questions_df["Explanation"] = all_explanations

        # Calculate topic statistics
        topic_stats = (
            questions_df.groupby("Topic")
            .agg({"Question No": "count"})
            .rename(columns={"Question No": "Question Count"})
        )
        topic_stats["Percentage"] = (topic_stats["Question Count"] / len(questions_df) * 100).round(2)

        # Save to Excel
        with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
            questions_df.to_excel(writer, sheet_name="Question Categories", index=False)
            topic_stats.to_excel(writer, sheet_name="Topic Statistics")

        print(f"Topic classification completed and saved to {output_path}")
        return True

    except Exception as e:
        print(f"Fatal error in topic classification: {str(e)}")
        return False

def start_processing(
    dialog,
    doc,
    last_question,
    current_student_name,
    csv_file_path,
    xlsx_file_path,
    file_directory,
    start_page=1,
):
    # This function will be run in a new thread
    try:
        jsonl_file_path = csv_file_path.replace(".csv", ".jsonl")
        process_pdf(
            dialog,
            doc,
            last_question,
            current_student_name,
            csv_file_path,
            jsonl_file_path,
            start_page,
            dialog.stop_page,
        )

        # Add topic categorization
        print("Starting topic categorization...")
        # Read the processed data
        data = pd.read_csv(csv_file_path)
        print("DEBUG: Raw data head:")
        print(data.head(10))

        # Defensive: Check for column names and fix if needed
        # expected = [
        #     "Student Name", "Question No", "Question",
        #     "Answer", "Grading", "PageNumber", "ClassName", "SubjectName"
        # ]
        # if list(data.columns)[: len(expected)] != expected:
        #     data.columns = expected[: data.shape[1]]
        expected_cols = {'Question No', 'Question'}

        if not expected_cols.issubset(data.columns):
            print(f"Input file must contain columns: {expected_cols}")
            return False

        # Remove rows with missing Question No or Question
        clean_questions = (
            data.dropna(subset=["Question No", "Question"])
            .assign(**{
                "Question No": lambda df: df["Question No"]
                .apply(lambda x: str(int(x)) if pd.notnull(x) and str(x).replace('.0','').isdigit() else str(x))
            })
        )
        

        # Drop duplicates just in case
        unique_questions = clean_questions[["Question No", "Question"]].drop_duplicates()
        print("DEBUG: unique_questions head:")
        print(unique_questions.head(10))


        # Generate topic classification output path
        topic_output_path = os.path.join(
            file_directory,
            f"topic_analysis_{dialog.subject_name}_{datetime.now().strftime('%d_%m_%Y_%H_%M')}.xlsx",
        )

        # Run topic categorization
        if not categorize_questions_by_topic(dialog,unique_questions, topic_output_path, str(dialog.subject_name).strip()):
            print("Topic categorization failed, continuing with remaining processing...")
            

        print(f"another possibility for potential file to be ingested iss {csv_file_path}")

        current_time = datetime.now()
        formatted_time = current_time.strftime(
            "%d_%m_%Y_%H_%M"
        )  # Format as day_month_year_hour_minute

        # Construct filenames with the current date and time
        output_filename = f"transcription_output_{formatted_time}.csv"
        debug_filename = f"transcription_deletions_{formatted_time}.xlsx"

        output_path = os.path.join(file_directory, output_filename)
        print(f"The output is supposed to go {output_path}")

        debug_output_path = os.path.join(file_directory, debug_filename)
        print(f"The current student name is {debug_output_path}")

        process_transcribed_data(csv_file_path, output_path, debug_output_path)
        analyze_misconceptions(dialog, csv_file_path, output_path, xlsx_file_path)
    except Exception as e:
        print(f"Error in processing files: {str(e)}")
    finally:
        dialog.update_queue.put(("close",))


def process_transcribed_data(file_path, output_path, debug_output_path):
    # Load the data
    data = pd.read_csv(file_path, delimiter=",", header=None)
    print(f"Printing data to see format\n")

    data.columns = [
        "Name",
        "Question",
        "Text",
        "Answer",
        "Status",
        "ScanPageNo",
        "ClassName",
        "SubjectName",
    ]

    # # Add class and subject columns
    data["ClassName"] = data["ClassName"].str.replace(r"[\(\)\[\]\']", "", regex=True)
    data["SubjectName"] = data["SubjectName"].str.replace(
        r"[\(\)\[\]\']", "", regex=True
    )

    # Filter out rows with only one column
    data.dropna(subset=["Question"], inplace=True)

    # Delete rows where 'Question No' appears in the 'Question' column
    data = data[~data["Question"].str.contains("Question No", na=False)]

    # Convert ScanPageNo to integer
    data["ScanPageNo"] = data["ScanPageNo"].astype(int)

    # Function to correct and format the question number
    def correct_question_number(question_number):
        corrected = re.sub(r"(\d+)\.([a-z]+)\.([ivx]+)", r"\1(\2)(\3)", question_number)
        corrected = re.sub(r"(\d+)\.([a-z]+)", r"\1(\2)", corrected)
        corrected = re.sub(r"(\d+)\.\(([a-z])\)\.([ivx]+)", r"\1(\2)(\3)", corrected)
        corrected = re.sub(r"\.\(", "(", corrected)
        corrected = corrected.replace(" ", "")
        return corrected.strip()

    # Apply the correction function to the 'Question' column
    data["Question"] = data["Question"].apply(correct_question_number)

    # Standardize question number formats
    def standardize_question_number(question_number):
        question_number = re.sub(
            r"(\d+)([a-z])\((i{1,3}|iv|v{1,2})\)", r"\1(\2)(\3)", question_number
        )
        question_number = re.sub(r"(\d+)([a-z])", r"\1(\2)", question_number)
        return question_number

    data["Question"] = data["Question"].apply(standardize_question_number)

    # Remove rows which are essentially headers repeated or instructional content
    data = data[~data["Question"].str.contains("Question No", na=False)]
    data = data.dropna(subset=["Question"])

    # Save the cleaned data
    data.to_csv(output_path, sep="|", index=False)
    print(f"Data cleaned and saved to {output_path}")

    data.to_excel(output_path.replace(".csv", ".xlsx"), index=False)
    print(f"Data cleaned and saved to {output_path.replace('.csv', '.xlsx')}")

    # Example of handling matched rows for debugging purposes
    matched_df = data[data.duplicated(subset=["Name", "Question"], keep=False)]
    debug_output_path = debug_output_path
    matched_df.to_excel(debug_output_path, index=False)
    print(f"Debug file with matched rows saved to {debug_output_path}")


def analyze_misconceptions(dialog, file_path, output_path, xlsx_file_path):
    data = pd.read_csv(
        file_path,
        delimiter=",",
        names=[
            "Student Name",
            "Question No",
            "Question",
            "Answer",
            "Grading",
            "ScanPageNo",
            "ClassName",
            "SubjectName",
        ],
    )

    def get_common_misconception(question, wrong_answers, correct_answers_sample):
        # Ensure all answers are strings and handle NaNs
        wrong_answers = [
            str(answer) if pd.notna(answer) else "" for answer in wrong_answers
        ]
        wrong_answers_text = "\n".join(wrong_answers)
        correct_answers_sample = [
            str(answer) if pd.notna(answer) else "" for answer in correct_answers_sample
        ]
        # Remove duplicates by converting the list to a set and back to a list
        correct_answers_sample = list(set(correct_answers_sample))
        # Join the list into a single string with each element on a new line
        correct_answers_text = "\n".join(correct_answers_sample)

        prompt = (
            f"The following question had the given wrong answers. "
            f"Identify the common misconception from these answers and provide a paraphrased explanation. Not all the wrong answers are part of the common misconception; the common misconception is a theme repeated among the wrong answers, so it appears as a subset of the given answers.\n\n"
            f"**Question:** {question}\n\n"
            f"**Wrong Answers:**\n{wrong_answers_text}\n\n"
            f"Now, please analyze the wrong answers and output your result as a single JSON object with the following keys:\n"
            f'- "misconception": a brief description of the common misconception.\n'
            f'- "count": the number of times this misconception appears in the list of wrong answers.\n\n'
            f"**Example Output:**\n"
            f'{{"misconception": "The first president of Uganda is Museveni", "count": 5}}\n\n'
            f"Do not include any additional text or formatting; only output the JSON object as shown.\n\n"
            f"Below is a sample of correct answers. It should help guide you to evaluate what the misconceptions above are. Please note that an AI transcriber picked the data above (misconcpetions) and so it could have incorrectly added some to the list.\n"
            f"{correct_answers_text}"
        )

        # Print the prompt for debugging purposes
        print("\n Correct Answers set: as it is in the get common misconceptions file")
        print(correct_answers_sample)
        print(f"Processing {current_sub_question}/{total_sub_questions}")
        print("Prompt:")
        print(prompt)
        print("\n Correct Answers:")
        print(correct_answers_text)

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {OPENAI_API_KEY}",
        }
        payload = {
            "model": "gpt-4o-mini",
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 800,
        }

        response = requests.post(
            "https://api.openai.com/v1/chat/completions", headers=headers, json=payload
        )
        response.raise_for_status()  # Raise error if request failed
        response_json = response.json()

        # Print the response for debugging
        print(response_json)

        result = json.loads(response_json["choices"][0]["message"]["content"].strip())
        misconception = result.get("misconception", "")
        count = int(result.get("count", 0))
        return misconception, count

    # Function to extract the main question number

    def get_main_question_no(q_no):
        q_no = str(q_no)
        if "(" in q_no:
            return q_no.split("(")[0]
        return q_no

    # Function to extract the sub-question number (up to the second level)
    def get_sub_question_no(q_no):
        q_no = str(q_no)
        if "(" in q_no:
            parts = q_no.split("(")
            return parts[0] + "(" + parts[1].split(")")[0] + ")"
        return q_no

    # Read the data from the text file
    #     data = pd.read_csv(file_path, delimiter='|', names=['Student Name', 'Question No', 'Question', 'Answer', 'Grading','ScanPageNo'])
    # Add new columns for main and sub-question numbers
    data["Main Question No"] = data["Question No"].apply(get_main_question_no)
    data["Sub Question No"] = data["Question No"].apply(get_sub_question_no)

    # Group by 'Sub Question No'
    grouped = data.groupby(["Sub Question No"])

    # Initialize lists to store the results
    results = []

    # Get the total number of sub-question numbers
    total_sub_questions = len(grouped)
    current_sub_question = 0

    # Process each group
    for name, group in grouped:
        current_sub_question += 1
        dialog.update_analysis_progress(
            current_sub_question, total_sub_questions
        )  # Update new progress label
        print(f"Processing {current_sub_question}/{total_sub_questions}")

        sub_question_no = name
        main_question_no = group["Main Question No"].iloc[0]
        question_text = group["Question"].iloc[0]  # Take the first question text
        attempts = len(group)
        distinct_students = group["Student Name"].nunique()
        correct_answers = (group["Grading"] == "Correct").sum()
        correct_percentage = (correct_answers / attempts) * 100 if attempts > 0 else 0

        # Get the most common misconception from wrong answers using GPT-3.5-turbo
        wrong_answers = group[
            (group["Grading"] != "Correct")
            & (pd.notna(group["Answer"]))
            & (group["Answer"].str.strip() != "")
        ]["Answer"].tolist()

        try:
            correct_answers_sample = (
                group[
                    (group["Grading"].str.contains("correct", case=False, na=False))
                    & (pd.notna(group["Answer"]))
                    & (group["Answer"].str.strip() != "")
                ]["Answer"]
                .sample(n=min(10, correct_answers), random_state=1)
                .tolist()
            )
            print("correct_answers_sample just after the set is created")
            print(correct_answers_sample)
        except ValueError:
            correct_answers_sample = ["No correct answer transcribed"]

        if wrong_answers:
            common_misconception, misconception_frequency = get_common_misconception(
                question_text, wrong_answers, correct_answers_sample
            )
        else:
            common_misconception = None
            misconception_frequency = 0

        results.append(
            [
                main_question_no,
                question_text,
                sub_question_no,
                attempts,
                distinct_students,
                correct_answers,
                f"{correct_percentage:.1f}",
                common_misconception,
                misconception_frequency,
            ]
        )

    # Create a DataFrame from the results
    results_df = pd.DataFrame(
        results,
        columns=[
            "Main Question No",
            "Question",
            "Sub Question No",
            "Attempts",
            "Distinct Students",
            "Correct Answers",
            "Correct %",
            "Most Common Misconception",
            "Misconception Frequency",
        ],
    )

    # Write the results to an Excel file
    # Final_report_path = 'C:/Users/maram/Downloads/Python projects/Optical Character recognition/Exam_Report.xlsx'
    with pd.ExcelWriter(xlsx_file_path) as writer:
        results_df.to_excel(writer, index=False, sheet_name="Questions_list")

    print(f"Report generated successfully: {xlsx_file_path}")


def get_file_selections_and_process():
    """Run the file selection dialog and process the PDF."""
    root = tk.Tk()
    dialog = FileSelectionDialog(root)
    root.mainloop()  # Start the Tkinter event loop

    if dialog.pdf_path and dialog.report_path:
        csv_file_path = dialog.report_path.replace(".xlsx", ".csv")
        xlsx_file_path = dialog.report_path
        file_directory = dialog.report_directory
        print(f"Selected PDF Path: {dialog.pdf_path}")
        print(f"Selected Report Path: {dialog.report_path}")
        print(f"Class: {dialog.class_name}")
        print(f"Subject: {dialog.subject_name}")
        print(f"Starting from page: {dialog.start_page}")

        doc = fitz.open(dialog.pdf_path)
        current_student_name = ""  # Initialize outside the loop to retain across pages
        Prev_last_question_no = (
            "N/A"  # Initialize outside the loop to retain across page
        )
        last_question = "N/A"  # Initialize outside the loop to retain across page

        # Process the PDF
        threading.Thread(
            target=start_processing,
            args=(
                dialog,
                doc,
                last_question,
                current_student_name,
                csv_file_path,
                xlsx_file_path,
                file_directory,
                dialog.start_page,
            ),
        ).start()

    else:
        print("No valid selections. Process cancelled.")
    root.mainloop()  # This should be at the end


# Run the application
get_file_selections_and_process()

In [ ]:
def robust_process_image_old(base64_image, last_known_student_name, retries=6, delay=16):
    """Attempt to process the image with retries upon failure, including handling specific API errors."""
    attempt = 0
    while attempt < retries:
        response_content = None  # Initialize here to ensure it's defined even on error
        try:
            response_content = process_image(base64_image, last_known_student_name)
            # Ensure the response is in the new JSON schema format:
            print("Response keys:", list(response_content.keys()))
            if not (
                isinstance(response_content, dict)
                and "studentName" in response_content
                and "entries" in response_content
            ):
                raise KeyError("Response does not conform to the expected JSON schema.")
            # If we have a valid response, return it.
            if response_content:
                return response_content

        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"Request failed with error: {e}. Retrying...")
            time.sleep(delay)
            attempt += 1

        except (
            KeyError
        ) as e:  # Catch errors in case the response JSON does not contain expected keys
            print("Printing prompt response below:")
            print(response_content)  # Now this variable is defined (even if None)
            print("Printed prompt response above:")
            print(
                f"Key error: {e} - likely missing or misformatted data in response.jiba jaba Retrying..."
            )
            time.sleep(delay)
            attempt += 1

        else:
            try:
                # Check for specific error messages indicating unsupported content
                if (
                    "error" in response_content
                    and response_content["error"]["message"]
                    == "The model produced invalid content. Consider modifying your prompt if you are seeing this error persistently."
                ):
                    print("Model produced invalid content error received. Retrying...")
                    time.sleep(delay)
                    attempt += 1
                elif (
                    "error" in response_content
                    and response_content["error"]["message"]
                    == "You uploaded an unsupported image. Please make sure your image is below 20 MB in size and is of one the following formats: ['png', 'jpeg', 'gif', 'webp']."
                ):
                    print(
                        "Unsupported image format or size error received. Retrying..."
                    )
                    time.sleep(delay)
                    attempt += 1
            except (TypeError, KeyError):
                # Handles scenarios where the response is not in the expected format
                print("Unexpected response format or missing data. Retrying...")
                time.sleep(delay)
                attempt += 1

    return None

In [ ]:
def process_pdf_old(
    dialog,
    doc,
    last_question,
    current_student_name,
    csv_file_path,
    jsonl_file_path,
    start_page=1,
    stop_page=None,
):
    start_idx = start_page - 1
    stop_idx = stop_page if stop_page else len(doc)
    original_file_name = os.path.basename(dialog.pdf_path).replace(".pdf", "")

    with open(csv_file_path, mode="w", encoding="utf-8") as csv_file, open(
        jsonl_file_path, mode="w", encoding="utf-8"
    ) as jsonl_file:

        # Write the CSV header row
        csv_file.write(
            "Student Name,Question No,Question,Answer,Grading,PageNumber,ClassName,SubjectName\n"
        )

        num_pages_to_process = min(stop_idx, dialog.max_pages)
        className = str(dialog.class_name).strip()
        subjectName = str(dialog.subject_name).strip()

        for page_num in range(start_idx, num_pages_to_process):
            dialog.safe_update_progress(page_num, dialog.total_pages)
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            base64_image = encode_image_to_base64(pix)

            # Optionally save the image
            save_page_image(
                base64_image, original_file_name, page_num + 1, dialog.report_directory
            )

            print(f"Processing page {page_num + 1}/{dialog.total_pages}...")
            response_json = robust_process_image(base64_image, current_student_name)
            if response_json is None:
                print(f"Failed to process page {page_num + 1}. Skipping...")
                continue

            # Extract the student name and entries from the JSON response
            student_name_extracted = response_json.get(
                "studentName", current_student_name
            )
            entries = response_json.get("entries", [])

            # Update the current student name for subsequent pages if found
            if student_name_extracted:
                current_student_name = student_name_extracted

            # Optionally, print first and last question numbers
            first_question, last_question = get_question_numbers_from_json(entries)
            print(f"First question number: {first_question}")
            print(f"Last question number: {last_question}")

            # Process each entry
            if entries:
                for entry in entries:
                    question_no = entry.get("questionNo", "")
                    question = entry.get("question", "")
                    answer = entry.get("answer", "")
                    grading = entry.get("grading", "")
                    page_str = str(page_num + 1)

                    # Escape quotes for CSV
                    question_escaped = question.replace('"', '""')
                    answer_escaped = answer.replace('"', '""')
                    grading_escaped = grading.replace('"', '""')

                    # Wrap each field that can contain commas or quotes in double quotes
                    quoted_question = f'"{question_escaped}"'
                    quoted_answer = f'"{answer_escaped}"'
                    quoted_grading = f'"{grading_escaped}"'
                    quoted_class = f'"{className}"'
                    quoted_subject = f'"{subjectName}"'

                    # 1) Create the CSV row
                    csv_row = ",".join(
                        [
                            student_name_extracted,
                            question_no,
                            quoted_question,
                            quoted_answer,
                            quoted_grading,
                            page_str,
                            quoted_class,
                            quoted_subject,
                        ]
                    )
                    csv_file.write(csv_row + "\n")

                    # 2) Write JSON line
                    jsonl_obj = {
                        "studentName": student_name_extracted,
                        "questionNo": question_no,
                        "question": question,
                        "answer": answer,
                        "grading": grading,
                        "pageNumber": page_num + 1,
                        "className": className,
                        "subjectName": subjectName,
                    }
                    jsonl_file.write(json.dumps(jsonl_obj) + "\n")
            else:
                # No entries, write at least a student name placeholder
                csv_file.write(student_name_extracted + ",,,,\n")
                jsonl_file.write(
                    json.dumps({"studentName": student_name_extracted}) + "\n"
                )